In [1]:
import urllib.request 

urllib.request.urlretrieve("https://www.dropbox.com/s/4xqg32ih9xoh5jq/UNSW-NB15.csv?dl=1", "UNSW-NB15.csv")

('UNSW-NB15.csv', <http.client.HTTPMessage at 0x7f8b28614490>)

In [2]:
unsw_rdd = sc.textFile("./UNSW-NB15.csv")

In [4]:
def AddNormalLabel(line): 
    line = line.split(",") 
    if line[47] == "": 
        line[47] = 'normal' 

    return line

In [9]:
# convert empty value to 'normal' in col. 47 
unsw_update_rdd = unsw_rdd.map(AddNormalLabel)
unsw_update_rdd.take(2)

[['59.166.0.3',
  '56716',
  '149.171.126.8',
  '143',
  'tcp',
  'FIN',
  '0.82546002',
  '7812',
  '16236',
  '31',
  '29',
  '30',
  '32',
  '-',
  '75090.25',
  '156111.73',
  '122',
  '126',
  '255',
  '255',
  '2751097753',
  '2748686736',
  '64',
  '129',
  '0',
  '0',
  '445.25928',
  '474.9451',
  '1421970774',
  '1421970775',
  '6.8190908',
  '6.599896',
  '5.9700001E-4',
  '4.6899999E-4',
  '0.000128',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2',
  '7',
  '1',
  '4',
  '1',
  '1',
  '1',
  'normal',
  '0'],
 ['59.166.0.0',
  '43467',
  '149.171.126.6',
  '49729',
  'tcp',
  'FIN',
  '0.101815',
  '4238',
  '65628',
  '31',
  '29',
  '7',
  '30',
  '-',
  '328438.84',
  '5087030.5',
  '72',
  '74',
  '255',
  '255',
  '961515433',
  '3225510659',
  '59',
  '887',
  '0',
  '0',
  '0',
  '91.579567',
  '1421970775',
  '1421970775',
  '1.429493',
  '1.387192',
  '0.00068',
  '5.4600002E-4',
  '0.000134',
  '0',
  '0',
  '0',
  '0',
  '0',
  '7',
  '4',
  '1',
  '6',
  '1',
  '1',
 

In [10]:
features=['srcip','sport','dstip','dsport','proto','state','dur','sbytes','dbytes','sttl','dttl','sloss',
          'dloss','service','Sload','Dload','Spkts','Dpkts','swin','dwin','stcpb','dtcpb','smeansz',
          'dmeansz','trans_depth','res_bdy_len','Sjit','Djit','Stime','Ltime','Sintpkt','Dintpkt','tcprtt',
          'synack','ackdat','is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd',
          'ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm','ct_src_dport_ltm','ct_dst_sport_ltm',
          'ct_dst_src_ltm','attack_cat','Label'] 
df_unsw = spark.createDataFrame(unsw_update_rdd, features)

In [11]:
#descriptive analysis

df_unsw.select('sbytes').describe().show()

+-------+------------------+
|summary|            sbytes|
+-------+------------------+
|  count|           2539739|
|   mean| 4340.072263330996|
| stddev|56409.398122862396|
|    min|                 0|
|    max|              9999|
+-------+------------------+



In [12]:
#convert features to int/float

from pyspark.sql.functions import col 
Desired_features = ['dbytes', 'sload', 'spkts', 'dpkts'] 
df_unsw_cast = df_unsw.select(*(col(c).cast('float').alias(c) for c in Desired_features))

In [14]:
#approximate quantiles 
Q1 = df_unsw_cast.approxQuantile('sload', [0.25], 0) 
Q2 = df_unsw_cast.approxQuantile('sload', [0.5], 0) 
Q3 = df_unsw_cast.approxQuantile('sload', [0.75], 0) 

print([Q1, Q2, Q3])

[[135376.25], [589303.75], [2038365.375]]


In [15]:
from pyspark.sql import functions as f 
df_unsw_cast.select(f.skewness(df_unsw_cast['dbytes']), f.skewness(df_unsw_cast['dpkts'])).show()

+-----------------+-----------------+
| skewness(dbytes)|  skewness(dpkts)|
+-----------------+-----------------+
|9.088530888998685|8.483613608319839|
+-----------------+-----------------+



In [16]:
df_unsw_cast.select(f.kurtosis(df_unsw_cast['spkts']), f.kurtosis(df_unsw_cast['dbytes'])).show()

+------------------+-----------------+
|   kurtosis(spkts)| kurtosis(dbytes)|
+------------------+-----------------+
|2265.4430659202717|261.8368671169627|
+------------------+-----------------+

